In [1]:
import celmech as cm
import numpy as np
import rebound as rb
import sympy as sp

In [2]:
from celmech.miscellaneous import truncated_expansion

Define the inertia tensor with coordinates in the body principal axes frame

In [3]:
# intertia tensor
delta,epsilon,C = sp.symbols("delta,epsilon,C")
A = C/(1+delta)
B = C/(1-delta)
C1 = C/(1-epsilon)
body_frame_intertia_tensor = sp.diag(A,B,C1)

body_frame_intertia_tensor

⎡  C                ⎤
⎢─────    0      0  ⎥
⎢δ + 1              ⎥
⎢                   ⎥
⎢         C         ⎥
⎢  0    ─────    0  ⎥
⎢       1 - δ       ⎥
⎢                   ⎥
⎢                C  ⎥
⎢  0      0    ─────⎥
⎣              1 - ε⎦

Define the free body Hamiltonian in terms of Andoyer variables 

In [4]:
# Andoyer variables
SZ,S,Lmbda = sp.symbols("S_z,S,Lambda",positive=True)
h,g,l = sp.symbols("h,g,l",real=True)

from sympy import sin,cos,acos

# Hamiltonian of torque-free rigid body in Andoyer variables
Hfree = Lmbda * Lmbda / (2 * C1) 
Hfree += (sin(l)**2/A + cos(l)**2/B)/2 * (S*S - Lmbda*Lmbda)

# some functions for performing specific simplifications of trig functions
from sympy.simplify.fu import TR8,TR10,TR10i,TR11

Hfree = sp.simplify(TR8(Hfree))

Hfree

 2               2      2               2
Λ ⋅δ⋅cos(2⋅l) - Λ ⋅ε - S ⋅δ⋅cos(2⋅l) + S 
─────────────────────────────────────────
                   2⋅C                   

Deine rotation matrices

In [5]:
def R1(x):
    s,c = sin(x),cos(x)
    R=sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
    return R
def R3(x):
    s,c = sin(x),cos(x)
    R=sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
    return R
def R313(alpha,beta,gamma):
    return R3(alpha)*R1(beta)*R3(gamma)

# Rotation matrices in terms of complex variables; 
#  (not currently used)
def R1complex(z,zbar):
    c = (z+zbar)/2
    s = (z-zbar)/2/sp.I
    R=sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
    return R
def R3complex(z,zbar):
    c = (z+zbar)/2
    s = (z-zbar)/2/sp.I
    R=sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
    return R
def R313complex(alpha,alphabar,beta,betabar,gamma,gammabar):
    return R3complex(alpha,alphabar)*R1complex(beta,betabar)*R3complex(gamma,gammabar)

Transformation from body and orbit frames to inertial reference frame. 
The components of $\mathsf{I}$ in the inertial frame are given by

$$
\begin{equation}
    I_{ij} = A_{il}A_{jm}[\mathrm{diag}(A,B,C)]_{lm} = A\cdot\mathrm{diag}(A,B,C)\cdot A^{T}~.\label{eq:I_inertial}
\end{equation}
$$

where $A = R_{3,1,3}(h,i,0) \cdot R_{3,1,3}(g,J,\ell)$.


The position vector of the planet is given by 
$$
\begin{equation}
r = a B \cdot \begin{pmatrix} \cos E - e \\ \sqrt{1-e^2}\sin E \\  0 \end{pmatrix}
\end{equation}
$$
with $B = R_{3,1,3}(\Omega, \arccos\vartheta,\omega)$

In [7]:
i = acos(SZ/S)
J = acos(Lmbda/S)

Amtrx = R313(h,i,0) * R313(g,J,l)
a,e =  sp.symbols("a,e",positive= True)
E,M,omega,Omega = sp.symbols("E,M,omega,Omega", real = True)
r_vec_orbit_frame = a * sp.Matrix([cos(E) - e, sp.sqrt(1-e*e)*sin(E),0])

vtheta = sp.symbols("vartheta", positive = True)
Bmtrx = R313(Omega,sp.acos(vtheta),omega)
r_vec_ref_frame = Bmtrx * r_vec_orbit_frame


In [8]:
Amtrx = R313(h,i,0) * R313(g,J,l)
# For principal axis rotation Lambda = S
Amtrx_PAR = R313(h,i,0) * R313(0,0,l) 

In [9]:
ref_frame_intertia_tensor = Amtrx_PAR * body_frame_intertia_tensor * Amtrx_PAR.transpose()

In [10]:
spin_av = lambda exprn: sp.integrate(exprn,(l,0,2*sp.pi))/(2*sp.pi)

In [11]:
ref_frame_intertia_tensor_av = spin_av(ref_frame_intertia_tensor)

In [12]:
r_vec_orbit_circ = r_vec_orbit_frame.xreplace({e:0})
r_vec_orbit_circ

⎡a⋅cos(E)⎤
⎢        ⎥
⎢a⋅sin(E)⎥
⎢        ⎥
⎣   0    ⎦

In [13]:
r_vec_ref_frame_circular =  sp.simplify(Bmtrx * r_vec_orbit_circ)

Get $\langle\mathsf{I}(\mathbf{r},\mathbf{r})\rangle$:

In [14]:
exprn = r_vec_ref_frame_circular.dot(
    (ref_frame_intertia_tensor * r_vec_ref_frame_circular)
)
exprn = exprn.expand()

In [15]:
exprn = exprn.func(*[sp.expand(TR8(a)) for a in exprn.args])

In [16]:
is_slow = lambda exprn : (E not in exprn.free_symbols) and (l not in exprn.free_symbols)

In [17]:
exprn_orbav = exprn.func(*[a for a in exprn.args if is_slow(a)])

In [18]:
exprn_orbav=sp.simplify(exprn_orbav)

In [19]:
exprn_orbav

     ⎛                                                                                                                                         
   2 ⎜   2  2         2                   2  2         2    2  2                   2  2    2           2                   2           2    2  
C⋅a ⋅⎝- S ⋅δ ⋅vartheta ⋅cos(2⋅Ω - 2⋅h) - S ⋅δ ⋅vartheta  + S ⋅δ ⋅cos(2⋅Ω - 2⋅h) - S ⋅δ  + S ⋅ε⋅vartheta ⋅cos(2⋅Ω - 2⋅h) + S ⋅ε⋅vartheta  - S ⋅ε
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                               
                                                                                                                                               

                                                                                                                                       